# Food Delivery Data Analysis

In [1]:
import pandas as pd 
import json
import re

In [2]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
with open("users.json") as f:
    users = pd.json_normalize(json.load(f))

users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
with open("restaurants.sql") as f:
    sql_data = f.read()

sql_data[:300]

"CREATE TABLE restaurants (\nrestaurant_id INT,\nrestaurant_name TEXT,\ncuisine TEXT,\nrating FLOAT);\n\nINSERT INTO restaurants VALUES (1, 'Restaurant_1', 'Chinese', 4.8);\nINSERT INTO restaurants VALUES (2, 'Restaurant_2', 'Indian', 4.1);\nINSERT INTO restaurants VALUES (3, 'Restaurant_3', 'Mexican', 4.3);"

In [5]:
rows = re.findall(
    r"\((\d+),\s*'[^']+',\s*'([^']+)',\s*([\d.]+)\)",
    sql_data
)

restaurants = pd.DataFrame(
    rows, columns=["restaurant_id", "cuisine", "rating"]
)

restaurants["restaurant_id"] = restaurants["restaurant_id"].astype(int)
restaurants["rating"] = restaurants["rating"].astype(float)

restaurants.head()


,restaurant_id,cuisine,rating
0,1,Chinese,4.8
1,2,Indian,4.1
2,3,Mexican,4.3
3,4,Chinese,4.1
4,5,Chinese,4.8


In [6]:
merged = orders.merge(users, on="user_id", how="left")


In [7]:
final_df = merged.merge(
    restaurants, on="restaurant_id", how="left"
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Italian,3.0


In [8]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


# Questions and Answers


# Which city has the highest total revenue (total_amount) from Gold members? 

In [10]:
# Filter only Gold member orders
gold_orders = final_df[final_df["membership"] == "Gold"]

# Calculate total revenue by city
gold_city_revenue = (
    gold_orders
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

# Display result
gold_city_revenue

gold_city_revenue.idxmax()


'Chennai'

# Which cuisine has the highest average order value across all orders?

In [11]:
# Calculate average order value by cuisine
avg_order_value_by_cuisine = (
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

# Display average order value for each cuisine
avg_order_value_by_cuisine

avg_order_value_by_cuisine.idxmax()


'Mexican'

# How many distinct users placed orders worth more than ₹1000 in total?

In [12]:
user_total_spend = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
)

(user_total_spend > 1000).sum()


np.int64(2544)

# Which restaurant rating range generated the highest total revenue?

In [13]:
bins = [0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=bins,
    labels=labels
)

rating_revenue = (
    final_df
    .groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

rating_revenue


C:\Users\Atharva\AppData\Local\Temp\ipykernel_5076\3823126737.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("rating_range")["total_amount"]


rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

# Among Gold members, which city has the highest average order value?

In [14]:
(
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

# Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [15]:
restaurant_count = (
    final_df
    .groupby("cuisine")["restaurant_id"]
    .nunique()
)

revenue_by_cuisine = (
    final_df
    .groupby("cuisine")["total_amount"]
    .sum()
)

pd.DataFrame({
    "restaurant_count": restaurant_count,
    "total_revenue": revenue_by_cuisine
}).sort_values("restaurant_count")

,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


# What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [16]:
gold_percentage = (
    final_df[final_df["membership"] == "Gold"].shape[0]
    / final_df.shape[0]
) * 100

round(gold_percentage)


50

Which restaurant has the highest average order value but less than 20 total orders?

In [17]:
restaurant_stats = (
    final_df
    .groupby("restaurant_name")
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
)

result = (
    restaurant_stats[restaurant_stats["total_orders"] < 20]
    .sort_values("avg_order_value", ascending=False)
)

result.head()


,total_orders,avg_order_value
restaurant_name,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895


Which combination contributes the highest revenue?

In [18]:
combo_revenue = (
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

combo_revenue


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

During which quarter of the year is the total revenue highest?

In [19]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)

quarter_revenue = (
    final_df
    .groupby(final_df["order_date"].dt.quarter)["total_amount"]
    .sum()
)

quarter_revenue


order_date
1    2010626.64
2    1945348.72
3    2037385.10
4    2018263.66
Name: total_amount, dtype: float64

How many total orders were placed by users with Gold membership?

In [20]:
final_df[final_df["membership"] == "Gold"].shape[0]


4987

What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

In [21]:
round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)


1889367

How many distinct users placed at least one order?

In [22]:
final_df["user_id"].nunique()


2883

What is the average order value (rounded to 2 decimals) for Gold members?

In [23]:
round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)


np.float64(797.15)

How many orders were placed for restaurants with rating ≥ 4.5?

In [24]:
final_df[final_df["rating"] >= 4.5].shape[0]


3374

How many orders were placed in the top revenue city among Gold members only?

In [25]:
gold_df = final_df[final_df["membership"] == "Gold"]

top_city = (
    gold_df
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

gold_df[gold_df["city"] == top_city].shape[0]


1337

In [26]:
orders.columns, users.columns


(Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
        'restaurant_name'],
       dtype='object'),
 Index(['user_id', 'name', 'city', 'membership'], dtype='object'))

In [27]:
"restaurants.sql"


'restaurants.sql'

In [28]:
final_df.shape[0]


10000

In [29]:
final_df.isna().sum()


order_id           0
user_id            0
restaurant_id      0
order_date         0
total_amount       0
restaurant_name    0
name               0
city               0
membership         0
cuisine            0
rating             0
rating_range       0
dtype: int64

In [30]:
pd.DataFrame.merge


<function pandas.core.frame.DataFrame.merge(self, right: 'DataFrame | Series', how: 'MergeHow' = 'inner', on: 'IndexLabel | AnyArrayLike | None' = None, left_on: 'IndexLabel | AnyArrayLike | None' = None, right_on: 'IndexLabel | AnyArrayLike | None' = None, left_index: 'bool' = False, right_index: 'bool' = False, sort: 'bool' = False, suffixes: 'Suffixes' = ('_x', '_y'), copy: 'bool | None' = None, indicator: 'str | bool' = False, validate: 'MergeValidate | None' = None) -> 'DataFrame'>

In [31]:
users.columns


Index(['user_id', 'name', 'city', 'membership'], dtype='object')

In [32]:
orders.columns, restaurants.columns


(Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
        'restaurant_name'],
       dtype='object'),
 Index(['restaurant_id', 'cuisine', 'rating'], dtype='object'))

In [33]:
restaurants.columns


Index(['restaurant_id', 'cuisine', 'rating'], dtype='object')

In [34]:
final_df["user_id"].value_counts().head()


user_id
2973    13
1515    12
874     11
496     11
1337    11
Name: count, dtype: int64